In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import tensorflow as tf 
import keras

Using TensorFlow backend.


In [8]:
#load data 
dataframe = pd.read_csv("data.csv") #dataframe 
dataframe = dataframe.drop(['index','price','sq_price'],axis = 1)
dataframe = dataframe[0:10] #using only 10 rows
dataframe

,area,bathrooms
0,2104.0,3.0
1,1600.0,3.0
2,2400.0,3.0
3,1416.0,2.0
4,3000.0,4.0
5,1985.0,4.0
6,1534.0,3.0
7,1427.0,3.0
8,1380.0,3.0
9,1494.0,3.0


In [11]:
#add label 
#1 is good buy 0 is bad buy
dataframe.loc[:,('y1')] = [1,1,1,0,0,1,0,1,1,1]
dataframe.loc[:,('y2')] = dataframe['y1'] == 0 
dataframe.loc[:,('y2')] = dataframe['y2'].astype(int) 
dataframe

,area,bathrooms,y1,y2
0,2104.0,3.0,1,0
1,1600.0,3.0,1,0
2,2400.0,3.0,1,0
3,1416.0,2.0,0,1
4,3000.0,4.0,0,1
5,1985.0,4.0,1,0
6,1534.0,3.0,0,1
7,1427.0,3.0,1,0
8,1380.0,3.0,1,0
9,1494.0,3.0,1,0


In [18]:
#prepare data for tensorflow 
#convert features into tensors 
inputX = dataframe.loc[:, ['area', 'bathrooms']].to_numpy()
inputY = dataframe.loc[:, ["y1", "y2"]].to_numpy()

In [19]:
inputX

array([[2.104e+03, 3.000e+00],
       [1.600e+03, 3.000e+00],
       [2.400e+03, 3.000e+00],
       [1.416e+03, 2.000e+00],
       [3.000e+03, 4.000e+00],
       [1.985e+03, 4.000e+00],
       [1.534e+03, 3.000e+00],
       [1.427e+03, 3.000e+00],
       [1.380e+03, 3.000e+00],
       [1.494e+03, 3.000e+00]])

In [20]:
inputY

array([[1, 0],
       [1, 0],
       [1, 0],
       [0, 1],
       [0, 1],
       [1, 0],
       [0, 1],
       [1, 0],
       [1, 0],
       [1, 0]], dtype=int64)

In [35]:
#write our hyperparameters 
learning_rate = 0.000001
training_epochs = 2000
display_steps = 50 
n_samples = inputY.size

In [36]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [37]:
#creating the computation graph also called as Neural Network 
#for feature input tensors , none means any number of examples 
#placeholders are gateaways for data into our computation graph 
x = tf.placeholder(tf.float32 , [None,2])

#create weights
#2x2 float weights , that will be keep updating thorughout the process
w = tf.Variable(tf.zeros([2,2]))

#adding biases 
b = tf.Variable(tf.zeros([2]))

# multiply our weights by our inputs 
# y = wx + b 
y_values = tf.add(tf.matmul(x,w),b)

#implementing softmax function which is our activation function 
y = tf.nn.softmax(y_values)

y_= tf.placeholder(tf.float32, [None,2])

In [38]:
#perform training 
#creating our cost function which is mean squared error
cost = tf.reduce_sum(tf.pow(y_ - y,2))/(2*n_samples)

#gradient descent 
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

In [40]:
#initialize variables and tensorflow sessions
init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [45]:
#training 
for i in range(training_epochs):
    sess.run(optimizer, feed_dict={x:inputX , y_:inputY})
    if i % display_steps == 0:
        cc = sess.run(cost,feed_dict={x:inputX , y_:inputY})
        print ("Training step:", '%04d' % (i), "cost=", "{:.9f}".format(cc))

print ("Optimization Finished!")
training_cost = sess.run(cost, feed_dict={x: inputX, y_: inputY})
print ("Training cost=", training_cost, "W=", sess.run(w), "b=", sess.run(b), '\n')

Training step: 0000 cost= 0.109537281
Training step: 0050 cost= 0.109537199
Training step: 0100 cost= 0.109537147
Training step: 0150 cost= 0.109537080
Training step: 0200 cost= 0.109537005
Training step: 0250 cost= 0.109536938
Training step: 0300 cost= 0.109536879
Training step: 0350 cost= 0.109536804
Training step: 0400 cost= 0.109536745
Training step: 0450 cost= 0.109536670
Training step: 0500 cost= 0.109536603
Training step: 0550 cost= 0.109536529
Training step: 0600 cost= 0.109536462
Training step: 0650 cost= 0.109536387
Training step: 0700 cost= 0.109536313
Training step: 0750 cost= 0.109536268
Training step: 0800 cost= 0.109536186
Training step: 0850 cost= 0.109536126
Training step: 0900 cost= 0.109536052
Training step: 0950 cost= 0.109535985
Training step: 1000 cost= 0.109535910
Training step: 1050 cost= 0.109535851
Training step: 1100 cost= 0.109535791
Training step: 1150 cost= 0.109535716
Training step: 1200 cost= 0.109535649
Training step: 1250 cost= 0.109535575
Training ste

In [46]:
#predictions 
sess.run(y, feed_dict={x: inputX })


array([[0.7112309 , 0.2887692 ],
       [0.66498965, 0.33501032],
       [0.73654455, 0.2634555 ],
       [0.64717317, 0.3528268 ],
       [0.7833244 , 0.21667556],
       [0.7006994 , 0.29930058],
       [0.6586662 , 0.3413338 ],
       [0.6482943 , 0.3517057 ],
       [0.6436931 , 0.3563069 ],
       [0.65480596, 0.345194  ]], dtype=float32)